# Delta Lake Lab 
## Unit 7: ZORDER & DATA SKIPPING

In the previous unit, we -
1. Learned how to time travel

In this unit, we will-
1. Learn about Z-Ordering and how it further optimizes data skipping

Z-Ordering is a (multi-dimensional clustering) technique to colocate related information in the same set of files. This co-locality is automatically used by Delta Lake in data-skipping algorithms. This behavior dramatically reduces the amount of data that Delta Lake on Apache Spark needs to read. To Z-Order data, you specify the columns to order on in the ZORDER BY clause.

Data skipping information is collected automatically when you write data into a Delta Lake table. Delta Lake takes advantage of this information (minimum and maximum values for each column) at query time to provide faster queries. You do not need to configure data skipping; the feature is activated whenever applicable. However, its effectiveness depends on the layout of your data. For best results, apply Z-Ordering.

### 1. Imports

In [1]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

22/10/22 23:41:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  885979867746


In [6]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"
print(DELTA_LAKE_DIR_ROOT)

gs://dll-data-bucket-885979867746/delta-consumable


In [7]:
# Lets take a look at the data lake before the zordering 
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part*

gs://dll-data-bucket-885979867746/delta-consumable/part-00000-0d0c8ea0-982f-4f67-ab9d-62f94e57db11-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-27993337-bc5b-4c93-9ab0-b77f48ac9160-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-293e9d10-a628-4cf0-b86c-f9f289913756-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-33f34593-184c-40b8-adfe-73facf9f043f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-5c57d030-c1f7-4f7b-b5d8-2100c4426482-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-670d672a-c41d-4391-a47e-90d45e589ac2-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-8572a416-efe4-47bf-ab8b-755973ae5a7a-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumab

The author's output was-
```
gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-228eb829-1144-4a6d-a0e2-4fd39d9e9f57-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-29d7051f-ea28-4bd9-a8fd-8d9f8e38f163-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-3eb98e04-4353-4e5f-a8a2-17a570111981-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-47c79787-64f2-453c-8474-52cbcaeef3c2-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-48902d8f-0572-4735-b0d7-95b1927cb294-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-598882f5-6a6b-489f-a551-6e782e67702f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-61eebddf-adb1-4088-9520-6a953e6d3fff-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-7b773887-eac1-481b-aa71-e57bd469f977-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-89dfe8e5-2d40-49f6-b0a0-4b320db62d14-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-b5620589-f197-4417-8fcb-ce013d82deb9-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-c36f0527-e995-41d2-a8ea-776cc865f816-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00001-487147d6-23f9-4a07-ae73-7cddbe8dfd06-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00001-8d9a7ec0-3a4d-43e4-8e33-379ffb2e4a3a-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00001-91311981-1505-4747-a8c9-6b3616efb4c0-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00001-ad3ddbfe-cec8-4877-8b05-3d40da1079ba-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00002-15ea05a6-24a6-41e3-b9d1-7073429eda5f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00002-1717bbe6-6038-404c-aea4-e83d93eb0fa8-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00002-d43dcdd6-5fb8-4b31-baa2-5458bf321285-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00003-4c2e181f-c1eb-458f-aa08-9be435e56bb5-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00003-c15a6560-6d08-41f9-9ff8-546a02d4fca7-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00003-e6f811e0-d076-4619-bb1f-4e1ffd7caeb0-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00004-7663ab3e-1fbb-4ef9-bb08-80d4e6e3d9f4-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00005-c9b6ff00-72cb-4264-8f2a-d01f6c27b759-c000.snappy.parquet
```

In [8]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

29


### 4. ZORDER

In [9]:
spark.sql("OPTIMIZE loan_db.loans_by_state_delta ZORDER BY (addr_state)").show(truncate=False)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used


+--------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|path                                              |metrics                                                                                                                                                                             |
+--------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|gs://dll-data-bucket-885979867746/delta-consumable|{1, 25, {8066, 8066, 8066.0, 1, 8066}, {397, 2069, 1585.48, 25, 39637}, 1, {all, {0, 0}, {25, 39637}, 0, {25, 39637}, 1, null}, 1, 25, 0, false, 0, 0, 1666482138456, 0, 8, 0, null}|
+--------------------------------------------------+------------

In [10]:
# Lets take a look at the data lake post the zordering. There is one extra file.  
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part*

gs://dll-data-bucket-885979867746/delta-consumable/part-00000-0d0c8ea0-982f-4f67-ab9d-62f94e57db11-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-27993337-bc5b-4c93-9ab0-b77f48ac9160-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-293e9d10-a628-4cf0-b86c-f9f289913756-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-33f34593-184c-40b8-adfe-73facf9f043f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-5c57d030-c1f7-4f7b-b5d8-2100c4426482-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-670d672a-c41d-4391-a47e-90d45e589ac2-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-8572a416-efe4-47bf-ab8b-755973ae5a7a-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumab

In [11]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

30


In [12]:
# Lets take a look at the transaction log post the zordering 
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000002.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000003.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000004.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000005.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000006.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000007.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000008.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/0000000000000000000

In [13]:
# And review what is in the delta log
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000011.json

{"add":{"path":"part-00000-b77f8967-b01f-4604-a3f3-3074163693e4-c000.snappy.parquet","partitionValues":{},"size":1967,"modificationTime":1666481881462,"dataChange":true,"stats":"{\"numRecords\":408,\"minValues\":{\"addr_state\":\"AK\",\"count\":0,\"collateral_value\":0.0},\"maxValues\":{\"addr_state\":\"WY\",\"count\":10134,\"collateral_value\":1.0134172359090886E8},\"nullCount\":{\"addr_state\":0,\"count\":0,\"collateral_value\":0}}"}}
{"add":{"path":"part-00001-3b7fbc64-5d70-4ea7-833e-783a2022ba14-c000.snappy.parquet","partitionValues":{},"size":1645,"modificationTime":1666481881554,"dataChange":true,"stats":"{\"numRecords\":153,\"minValues\":{\"addr_state\":\"AK\",\"count\":0,\"collateral_value\":0.0},\"maxValues\":{\"addr_state\":\"WY\",\"count\":3168,\"collateral_value\":3.1680389069416154E7},\"nullCount\":{\"addr_state\":0,\"count\":0,\"collateral_value\":0}}"}}
{"add":{"path":"part-00002-570c06cd-732f-494f-ab81-2732881e654f-c000.snappy.parquet","partitionValues":{},"size":1959,"

### THIS CONCLUDES THIS LAB. PROCEED TO THE NEXT NOTEBOOK.